# Diagnose oscillations during slews for SITCOMTN-081

This notebook identifies slews with hardpoint forces going above a certain threshold. Once these are identified, it allows to plot the hardpoint forces and mount movements.

### Prepare Notebook

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
%load_ext lab_black

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from astropy.time import Time

from lsst.summit.utils.tmaUtils import TMAEventMaker, TMAState
from lsst.summit.utils.efdUtils import getEfdData, makeEfdClient
from lsst_efd_client import EfdClient

import warnings

warnings.filterwarnings("ignore")

In [ ]:
# create a client to retrieve datasets in the EFD database
client = EfdClient("usdf_efd")

### Define relevant settings

#### Observation day

In [ ]:
## Insert here the dayObs of interest
dayObs = 20230627  # 20230627  # 20231220  # 20231129

### Define functions

#### get previous logged detailedState event from a given time stamp

The status of M1M3 is not persistent in the EFD. In order to get at a given time what is the current status, use this function.

In [ ]:
from lsst.ts.xml.enums.MTM1M3 import DetailedStates


def get_previous_logged_detailedState(df_state, timestamp):
    """
    Get logged detailedState from M1M3 immediately before arbitrary time
    Args:
       df_state (pandas dataframe): pandas dataframe obtained from  time series of
          "lsst.sal.MTM1M3.logevent_detailedState" covering a wide time frame which includes
          the time stamp
       timestamp (pandas timestamp): a timestamp where we want to probe the current status of M1M3
    Returns:
       prev_state: human readable status of current M1M3 status
    """
    df_state_names = df_state["detailedState"].map(lambda x: DetailedStates(x).name)
    previous_index = df_state.index.asof(timestamp)
    prev = df_state.index.get_loc(previous_index)
    # add functionality for checking the end
    return df_state_names[prev]

#### compare hardpoints vs mount data

This is an auxiliary function to plot information from the M1M3 hardpoints with respect to the elevation and azimuth measurements from the mount.

In [ ]:
def compare_mount_hardpoints(
    df_mtmount_ele,
    df_mtmount_azi,
    df_hp,
    begin,
    end,
):
    fig, axs = plt.subplots(3, 1, dpi=125, figsize=(6, 8))

    # ax = axs[0]
    # df_plot = df_hp["fx"][begin:end]
    # ax.plot(df_plot, color="red", lw="0.5", label="HP fx")
    # df_plot = df_hp["fy"][begin:end]
    # ax.plot(df_plot, color="blue", lw="0.5", label="HP fy")
    # df_plot = df_hp["fz"][begin:end]
    # ax.plot(df_plot, color="black", lw="0.5", label="HP fz")
    # ax.set_ylabel("HP Force \n[N]")
    ax = axs[0]
    df_plot = df_hp["measuredForce0"][begin:end]
    ax.plot(df_plot, color="red", lw="0.5", label="HP 0")
    df_plot = df_hp["measuredForce1"][begin:end]
    ax.plot(df_plot, color="blue", lw="0.5", label="HP 1")
    df_plot = df_hp["measuredForce2"][begin:end]
    ax.plot(df_plot, color="black", lw="0.5", label="HP 2")
    df_plot = df_hp["measuredForce3"][begin:end]
    ax.plot(df_plot, color="green", lw="0.5", label="HP 3")
    df_plot = df_hp["measuredForce4"][begin:end]
    ax.plot(df_plot, color="orange", lw="0.5", label="HP 4")
    df_plot = df_hp["measuredForce5"][begin:end]
    ax.plot(df_plot, color="yellow", lw="0.5", label="HP 5")
    ax.set_ylabel("HP Force \n[N]")

    ax = axs[1]
    df_plot = df_mtmount_ele["actualPosition"][begin:end]
    ax.plot(df_plot, color="red", lw="0.5")
    # ax.axvline(begin, lw="0.5", c="k", label="Slew start")
    # ax.axvline(end, lw="0.5", c="b", label="Slew stop")
    ax.set_ylabel("TMAElevation \nPosition\n[deg]")

    ax = axs[2]
    df_plot = df_mtmount_azi["actualPosition"][begin:end]
    ax.plot(df_plot, color="red", lw="0.5")
    ax.set_ylabel("TMA Azimuth \nPosition\n[deg]")

    ##ax = axs[3]
    # df_plot = df_mtmount_ele["actualTorque"][begin:end]
    # ax.plot(df_plot, color="red", lw="0.5")
    # ax.set_ylabel("TMA Elevation \nTorque\n[Nm]")

    # ax = axs[4]
    # df_plot = df_mtmount_azi["actualTorque"][begin:end]
    # ax.plot(df_plot, color="red", lw="0.5")
    # ax.set_ylabel("TMA Azimuth \nTorque\n[Nm]")

    ax.set_xlabel("UTC")
    fig.autofmt_xdate()
    fig.subplots_adjust(hspace=1)
    fig.suptitle(t0)
    fig.legend()
    fig.tight_layout()
    ### TBD: use a delta time wrt slew stop in x-label

### Load data

#### Get slews and tracks

In [ ]:
# Select data from a given date
eventMaker = TMAEventMaker()
events = eventMaker.getEvents(dayObs)

# Get lists of slew and track events
slews = [e for e in events if e.type == TMAState.SLEWING]
tracks = [e for e in events if e.type == TMAState.TRACKING]
print(f"There are {len(events)} events")
print(f"Found {len(slews)} slews and {len(tracks)} tracks")

In [ ]:
# Get slews passing certain criteria
slews_selected = []
hp_max_hist = np.empty(len(slews))
min_ele_range = 1  # minimum elevation change in slew, degrees
min_azi_range = 1  # minimum azimuth change in slew, degrees
hp_threshold = (
    900  # maximum tolerated hardpoint force, Newtons, this is 30% of the breakaway
)
df_state = getEfdData(
    client,
    "lsst.sal.MTM1M3.logevent_detailedState",
    begin=Time(slews[0].begin, format="isot", scale="utc"),
    end=Time(slews[-1].end, format="isot", scale="utc"),
)  # get an array for all state changes from first slew of dayObs to final one
for i, slew in enumerate(slews):
    if (
        slew.seqNum == 0
    ):  # skip first one to avoid problems looking for a previous detailedState outside the df_state range
        continue
    df_azi = getEfdData(client, "lsst.sal.MTMount.azimuth", event=slew)
    df_ele = getEfdData(client, "lsst.sal.MTMount.elevation", event=slew)
    df_hp = getEfdData(client, "lsst.sal.MTM1M3.hardpointActuatorData", event=slew)
    timestamp = pd.Timestamp(
        Time(slew.begin, format="iso", scale="utc").value, tz="utc"
    )
    begin_state = get_previous_logged_detailedState(df_state, timestamp)
    timestamp = pd.Timestamp(Time(slew.end, format="iso", scale="utc").value, tz="utc")
    end_state = get_previous_logged_detailedState(df_state, timestamp)
    if len(df_azi) > 0:
        slew_delta_azi = df_azi["demandPosition"].max() - df_azi["demandPosition"].min()
        slew_delta_ele = df_ele["demandPosition"].max() - df_ele["demandPosition"].min()
        # slew_condition = (slew_delta_azi == 0) and (slew_delta_ele > min_ele_range)
        slew_ele_condition = slew_delta_ele > min_ele_range
        slew_azi_condition = slew_delta_azi > min_azi_range
        # print(np.max(abs(df_hp["fx"].values)))
        hp_max_individual = np.array(
            [
                np.max(abs(df_hp["measuredForce0"].values)),
                np.max(abs(df_hp["measuredForce1"].values)),
                np.max(abs(df_hp["measuredForce2"].values)),
                np.max(abs(df_hp["measuredForce3"].values)),
                np.max(abs(df_hp["measuredForce4"].values)),
                np.max(abs(df_hp["measuredForce5"].values)),
            ]
        )
        hp_max = np.max(hp_max_individual)
        hp_max_hist[i] = hp_max
        hp_condition = (
            hp_max > hp_threshold
        )  # if maximum force above threshold, flag it
        state_condition = (  # ensure that the HPs are active (mirror is 'raised')
            (begin_state == "ACTIVE") or (begin_state == "ACTIVEENGINEERING")
        ) and ((end_state == "ACTIVE") or (end_state == "ACTIVEENGINEERING"))
        if (
            slew_ele_condition
            and slew_azi_condition
            and state_condition
            and hp_condition
        ):
            slews_selected.append(slew)
print(
    f"There are {len(slews_selected)} selected slews out of {len(slews)} with measured HP force instance > {hp_threshold} newtons"
)

In [ ]:
# Plot a distribution of maximum forces
plt.ylabel("Number of slews")
plt.xlabel("Maximum recorded force in hardpoints (N)")
plt.title(f"Maximum force on hardpoints for {dayObs}")
plt.hist(hp_max_hist, bins=100, range=[0, 2000])

In [ ]:
for i, slew in enumerate(slews_selected):
    print(
        Time(slew.begin, format="isot", scale="utc"),
        slew.seqNum,
        slew.endReason.name,
    )

In [ ]:
selected_event = slews[476]  # slews[31] on 20230627 is the continuous oscilation one
print(selected_event.seqNum)
print(selected_event.begin)
postPadding = 50.0
start_slew = Time(selected_event.begin, format="isot", scale="utc")  # start of slew
t0 = pd.to_datetime(start_slew.value, utc=True)  # astropy Time to Timestamp conversion
end_slew = Time(selected_event.end, format="isot", scale="utc")  # end of slew
t1 = pd.to_datetime(end_slew.value, utc=True)  # astropy Time to Timestamp conversion
print("Slew start at:", t0)
print("Slew stop at:", t1)
print("type", selected_event.type.name)
print("end reason:", selected_event.endReason.name)
timestamp = pd.Timestamp(
    Time(selected_event.begin, format="iso", scale="utc").value, tz="utc"
)
print(get_previous_logged_detailedState(df_state, timestamp))

# Get mount data
df_mtmount_ele = getEfdData(
    client,
    "lsst.sal.MTMount.elevation",
    begin=start_slew,
    end=end_slew,
)
df_mtmount_azi = getEfdData(
    client,
    "lsst.sal.MTMount.azimuth",
    begin=start_slew,
    end=end_slew,
)
df_hp = getEfdData(
    client, "lsst.sal.MTM1M3.hardpointActuatorData", begin=start_slew, end=end_slew
)

### Look at data


In [ ]:
%matplotlib inline
compare_mount_hardpoints(
    df_mtmount_ele,
    df_mtmount_azi,
    df_hp,
    t0,
    t1,
)